In [1]:
import numpy as np
import hsbalance as hs

Using the Community license in this session. If you have a full Xpress license, first set the XPAUTH_PATH environment variable to the full path to your license file, xpauth.xpr, and then restart Python. If you want to use the FICO Community license and no longer want to see this message, set the XPAUTH_PATH environment variable to: /home/maged/anaconda3/envs/balance/lib/python3.8/site-packages/xpress/license/community-xpauth.xpr
NB: setting XPAUTH_PATH will also affect any other Xpress products installed on your system.


# Introduction

In these notes, we are going to discuss the issues related to robust optimization.
a. What is Robust Optimization?
Optimization process is not determinant, this means that we have no clue or determinant check if our solution is the optimum one. Optimization is prone to error if outliers exist. Noisy data and uncertainty readings. Methods, especially least squares method, can be more effected with outliers than others.  
b. Rotor Balancing Robust Optimization:
In our problem of rotor balancing, solution can be greatly effected by outliers for the following reasons:  
1. Measurements are low in numbers: Maximum field measuring points number is still tiny mathematically-wise.
2. Sensor faults: a sensor can go wrong easily for a turbine that runs for 20 years. Calibration is not an easy or a cheap process.  
3. Accumulation of errors: The process of measuring a point in a turbine/compressor usually involves a sensor and long cables in series (vibration cards and junctions). Those produce errors in series and cause the summation to infiltrate.
4. Measuring times: Turbo-Machinery is often connected to large process, which make it start and stop time costly. This encourages for accurate balancing with the fewest number of starts.

# Simulating Data

We generate random data with known solution W

In [2]:
np.random.seed(42)
real = np.random.rand(5,3)
imag = np.random.rand(5,3)
alpha = real + imag * 1j
# Assume W is equal to 10 grams @ 0 degree at all planes
W = [[10], [10], [10]]
# Create intial condtioning column array A with random noise to simulate actual problem
A = - alpha @ W + np.random.normal(1, scale=0.1, size=(5,1))

In [3]:
# Error due to generated noise
hs.rmse(alpha @ W + A)

0.9546

# Create a hsbalance model

In [4]:
model_alpha = hs.Alpha()
model_alpha.add(alpha)
model = hs.LeastSquares(A, model_alpha)
W_model = model.solve()
hs.convert_cart_math(W_model)

array([['9.448 @ 4.1'],
       ['9.49 @ 0.1'],
       ['9.842 @ 1.5']], dtype='<U11')

Calculate the Error

In [5]:
hs.rmse(W_model - W)

0.5666

# Faulty Sensor

To simulate a faulty sensor we will assume that the sensor at the first plane has gone completely fault and gives no output (i.e. equal 0)

In [6]:
A[0] =0
model_alpha = hs.Alpha()
model_alpha.add(alpha)
model = hs.LeastSquares(A, model_alpha)
W_model = model.solve()
hs.convert_cart_math(W_model)

array([['11.739 @ 18.8'],
       ['5.966 @ 294.0'],
       ['7.123 @ 35.6']], dtype='<U13')

In [7]:
hs.rmse(W_model - W)

6.3969

As we see, the error has gone extremly high and obviously unacceptable.  
If we acknowldge the sensor error, we can use weight least squares method to solve the problem instead.  
This can be done by assign the argument solver='WLS' in solver method of least squares. and pass the coefficient column array C to the model.   
In our case we will give all the sensor the same weight as 1. Only the sensor suspected we will assume a weight of 0. The previous setup will completely discard the sensor from caluclations.

In [8]:
C = np.ones(A.shape)
C[0] = 0
model = hs.LeastSquares(A=A, alpha=model_alpha, C=C)
W_model = model.solve(solver='WLS')
hs.convert_cart_math(W_model)

array([['9.52 @ 4.1'],
       ['9.317 @ 0.4'],
       ['9.865 @ 2.1']], dtype='<U11')

In [9]:
hs.rmse(W_model - W)

0.6433

We can notice that we restored a low level of error when discarding the faulty sensor.

# Huber Optimization

The previous example assumes we know for sure the defected sensor, so we down-weighted it to decrease the leaking noise it causes. A more convenient method but more conservative. The method make the model less sensitive to outliers.

In [10]:
model = hs.LeastSquares(A, model_alpha)
W_model = model.solve(solver='huber')
hs.convert_cart_math(W_model)

array([['10.859 @ 19.6'],
       ['7.188 @ 316.5'],
       ['6.926 @ 16.3']], dtype='<U13')

In [11]:
hs.rmse(W_model - W)

4.8004

The error improved but in a more conservative way.

| Method | RMSE |
|--|--|
|Least_squares|0.5666|
|Least_squares with errorous sensor|6.3969|
|Weighted_Least_squares|0.6433|
|Huber_Robust|4.8004|